In [1]:
from numba import cuda, float32, int32
import numpy as np
import os
import time
import copy
import operator
import math

# This next part is to avoid installing any packages. 
# On my machines I have an environment variable "LDPC" which contains the path to the project directory
# If you want a quick fix, you can comment the next line and set projectDir to where you cloned the repo.
projectDir = os.environ.get('LDPC')
if projectDir == None:
    import pathlib
    projectDir = pathlib.Path(__file__).parent.absolute()

# This part inserts the project directory into the environment path for the duration of this application.
import sys
sys.path.insert(1, projectDir)

# Import helper modules
import fileHandler
import common

# Next we set up a local prng, this is because we use np.random in several places, and we want each to have their own seed.
LDPC_LOCAL_PRNG = np.random.RandomState(7134066)
CODEWORD_LENGTH = 8176
# Now let's import the baseline matrix, I.e.: a matrix which is actually a good ECC
nearEarthParity = np.int32(fileHandler.readMatrixFromFile(str(projectDir) + '/codeMatrices/nearEarthParity.txt', 1022, 8176, 511, True, False, False))


In [2]:
#If everything went ok, then you should now be able to import ldpcCUDA.py
import ldpcCUDA



In [ ]:
# As a sanity check, I usually call this function:
ldpcCUDA.testNearEarth(numOfTransmissions = 10, graphics = True)
# It would take a few minutes to finish, depending on the value of numberOfTransmissions.
# With respect to our discussion, numberOfTransmissions is the number of samples for each sample of s^2
# The output should be how much time it took (so for example 19 seconds on my machine with 30 transmissions) 
# and throughput (37693 bits per second on my machine). 
# The higher the throughput, the more samples we can take.

In [7]:
# Now let's see how to generate epsilons:

SNRdb = 3.6
SNR = 10 ** (SNRdb/10)
## Now use the definition: SNR = signal^2 / sigma^2
sigma = np.sqrt(0.5 / SNR)
# This is where we sample epsilon
epsilon = np.float32(LDPC_LOCAL_PRNG.normal(0, sigma, CODEWORD_LENGTH))
sigmaActual = np.sqrt((np.sum(epsilon ** 2)) / CODEWORD_LENGTH)
print(sigma)
print(sigmaActual)
# Now, you may want to know, if we just tried to take epsilon and do no ECC, how many "hard" errors are there.
# For that we first "slice" epsilon (i.e.: if epsilon[i] <= 0 then slicedEpsilon = 0, otherwise slicedEpsilon[i] = 1)

0.46718081737169276
0.46571935680809146


In [8]:
berUncoded, berDecoded, iterator, totalTime = ldpcCUDA.evaluateMatrixAndEpsilon(nearEarthParity, epsilon, numberOfIterations = 50, cudaDeviceNumber = 0) #You can also set the cuda device number using, the default is 0

Using cuda device number: 0
*** while iterator finished at 7
0.0


In [14]:
print("Without any error correction the bit error rate is %d / <length of codeword> == %f" % (berUncoded, berUncoded/8176))
print("After %d iterations of min-sum decoding, the bit error rate is %d / <length of codeword> == %f" % (iterator, berDecoded, berDecoded/8176))

Without any error correction the bit error rate is 160 / <length of codeword> == 0.019569
After 7 iterations of min-sum decoding, the bit error rate is 0 / <length of codeword> == 0.000000


In [24]:
# Now let's check the variability 